In [31]:
import pandas as pd

myfile="8_new_data.csv"
mydata = pd.read_csv(myfile)
print(mydata.info())
print(mydata.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1473 entries, 0 to 1472
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   diary_entry  1473 non-null   object
 1   joy          1473 non-null   int64 
 2   sadness      1473 non-null   int64 
 3   anger        1473 non-null   int64 
 4   fear         1473 non-null   int64 
 5   disgust      1473 non-null   int64 
 6   surprise     1473 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 80.7+ KB
None
                                         diary_entry  joy  sadness  anger  \
0  My family was the most salient part of my day,...    0        0      0   
1  Yoga keeps me focused. I am able to take some ...    0        0      0   

   fear  disgust  surprise  
0     0        0         0  
1     0        0         0  


In [ ]:
import os
import re
import json
os.environ["OPENAI_API_KEY"] = "#####"

from openai import OpenAI
client = OpenAI()

def get_json(content):
    #get json part
    json_text = re.search(r'\{.*\}', content, re.DOTALL)
    # check json data
    if json_text:
        myjson = json_text.group()
        #print("Extracted JSON:", myjson)  # check extracted json
        try:
            data = json.loads(myjson)  # JSON parsing
            return data
        except json.JSONDecodeError as e:
            print("[jp] JSON parsing error:", e)
    else:
        print("[jp] JSON data not found")
    
    
    

def analyze_emotions(system_content,user_content):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ]
    )

    answer = response.choices[0].message.content #get json response
    return get_json(answer)

system_content = """
You will analyze the diary entry based on the 6 core emotions in psychology: joy, sadness, anger, fear, disgust, surprise. Each emotion's intensity will be rated on a scale of 1 to 5 as follows:
1 - very weak (barely felt)
2 - weak (slightly felt)
3 - moderate (moderately felt)
4 - strong (strongly felt)
5 - very strong (very strongly felt)
Please respond strictly in JSON format with the following structure:

{
  "diary_entry": "...",
  "joy": ...,
  "sadness": ...,
  "anger": ...,
  "fear": ...,
  "disgust": ...,
  "surprise": ...
}
"""

user_content="""
Diary_entry="It is 8pm and I have no idea when I am going home. I also need to check my parking meter."
"""

#json_result = analyze_emotions(system_content,user_content)
#print(json_result)
#print(json_result['anger'])

In [33]:
# 각 감정의 컬럼 이름
emotion_columns = ["joy", "sadness", "anger", "fear", "disgust", "surprise"]

# 각 diary_entry에 대해 감정 분석을 수행하고 결과 업데이트
for index, row in mydata.iterrows():
    print(f"Index: {index}")
    
    # 감정 분석 함수 호출
    json_result = analyze_emotions(system_content, row['diary_entry'])
    
    # JSON 결과에서 각 감정의 강도를 추출하여 data DataFrame에 업데이트
    for emotion in emotion_columns:
        if emotion in json_result:
            mydata.at[index, emotion] = json_result[emotion]
        else:
            # json_result에 해당 감정이 없을 경우 기본값으로 0 설정
            mydata.at[index, emotion] = 0

# 업데이트된 DataFrame 확인
print(mydata.head())
# 업데이트된 DataFrame을 CSV 파일로 저장
mydata.to_csv('updated_data.csv')

Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16
Index: 17
Index: 18
Index: 19
Index: 20
Index: 21
Index: 22
Index: 23
Index: 24
Index: 25
Index: 26
Index: 27
Index: 28
Index: 29
Index: 30
Index: 31
Index: 32
Index: 33
Index: 34
Index: 35
Index: 36
Index: 37
Index: 38
Index: 39
Index: 40
Index: 41
Index: 42
Index: 43
Index: 44
Index: 45
Index: 46
Index: 47
Index: 48
Index: 49
Index: 50
Index: 51
Index: 52
Index: 53
Index: 54
Index: 55
Index: 56
Index: 57
Index: 58
Index: 59
Index: 60
Index: 61
Index: 62
Index: 63
Index: 64
Index: 65
Index: 66
Index: 67
Index: 68
Index: 69
Index: 70
Index: 71
Index: 72
Index: 73
Index: 74
Index: 75
Index: 76
Index: 77
Index: 78
Index: 79
Index: 80
Index: 81
Index: 82
Index: 83
Index: 84
Index: 85
Index: 86
Index: 87
Index: 88
Index: 89
Index: 90
Index: 91
Index: 92
Index: 93
Index: 94
Index: 95
Index: 96
Index: 97
Index: 98
Index: 99
Index: 100

In [34]:
import pandas as pd

new_data = pd.read_csv("updated_data.csv")
print(new_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1473 entries, 0 to 1472
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1473 non-null   int64 
 1   diary_entry  1473 non-null   object
 2   joy          1473 non-null   int64 
 3   sadness      1473 non-null   int64 
 4   anger        1473 non-null   int64 
 5   fear         1473 non-null   int64 
 6   disgust      1473 non-null   int64 
 7   surprise     1473 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 92.2+ KB
None


In [36]:


# 각 열별로 값의 개수 확인
for column in new_data.columns:
    print(f"Column: {column}")
    print(new_data[column].value_counts())
    print("-" * 30)

Column: Unnamed: 0
Unnamed: 0
0       1
979     1
988     1
987     1
986     1
       ..
489     1
488     1
487     1
486     1
1472    1
Name: count, Length: 1473, dtype: int64
------------------------------
Column: diary_entry
diary_entry
My family was the most salient part of my day, since most days the care of my 2 children occupies the majority of my time. They are 2 years old and 7 months and I love them, but they also require so much attention that my anxiety is higher than ever. I am often overwhelmed by the care the require, but at the same, I am so excited to see them hit developmental and social milestones.                                                                                                                                                                                                                                 1
It was just a walk in the park with my dogs, but the air has started to change here, it is gloomy but charged, like it's just about to rain, and b

In [7]:
system_content = """
You will analyze the diary entry based on the 6 core emotions in psychology: joy, sadness, anger, fear, disgust, surprise. Each emotion's intensity will be rated on a scale of 1 to 5 as follows:
1 - very weak (barely felt)
2 - weak (slightly felt)
3 - moderate (moderately felt)
4 - strong (strongly felt)
5 - very strong (very strongly felt)
Please respond strictly in JSON format with the following structure:

{
  "diary_entry": "...",
  "joy": ...,
  "sadness": ...,
  "anger": ...,
  "fear": ...,
  "disgust": ...,
  "surprise": ...
}
"""

user_content="""
Diary_entry="My first ML class was pure misery. Nothing makes sense, and I feel utterly lost. Am I the only one who doesn’t understand this class? The professor seems to explain things well, but I feel like I won’t be able to finish this course successfully. I feel like I’ll have nightmares again tonight about not completing my ML assignment. Each day is just so stressful.
"""

json_result = analyze_emotions(system_content,user_content)
print(json_result)

{'diary_entry': 'My first ML class was pure misery. Nothing makes sense, and I feel utterly lost. Am I the only one who doesn’t understand this class? The professor seems to explain things well, but I feel like I won’t be able to finish this course successfully. I feel like I’ll have nightmares again tonight about not completing my ML assignment. Each day is just so stressful.', 'joy': 1, 'sadness': 4, 'anger': 2, 'fear': 4, 'disgust': 2, 'surprise': 2}


In [42]:
system_content = """
주어진 코드를 아이도 알정도로 쉽게 설명해줘.
"""

user_content="""
import torch
from sentence_transformers import SentenceTransformer, util

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 각 행을 텍스트로 변환하여 임베딩 생성
def create_table_embeddings(df, embedding_model):
    rows = df.apply(lambda row: " ".join(row.astype(str)), axis=1)
    row_embeddings = embedding_model.encode(rows, convert_to_tensor=True)
    return rows, row_embeddings

rows, row_embeddings = create_table_embeddings(mytable, embedding_model)

# 질문과 가장 유사한 행 찾기
def find_relevant_rows(question, row_embeddings, embedding_model, k=3):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)
    similarities = util.cos_sim(question_embedding, row_embeddings)
    top_k_indices = torch.topk(similarities, k=k).indices.flatten().tolist()
    return top_k_indices

# 질문에 답변하기 위한 함수
def answer_table_question(question, mytable, row_embeddings, embedding_model, k=3):
    top_k_indices = find_relevant_rows(question, row_embeddings, embedding_model, k)
    context = "\n".join([rows[i] for i in top_k_indices])
    print(f"Relevant Rows for Question: '{question}':\n{context}\n")

# 예시 질문
question = "Which product has the highest customer rating?"
answer_table_question(question, mytable, row_embeddings, embedding_model)
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": user_content
        }
    ]
)

answer = response.choices[0].message
print(answer)

ChatCompletionMessage(content='이 코드는 컴퓨터가 표(테이블)라는 데이터를 이해하고 질문에 대답할 수 있게 만들어요.\n\n1. **표를 이해시키기**:\n   - **표**는 여러 줄과 칸으로 된 데이터 덩어리예요. 예를 들어, 칸에는 날짜, 제품 이름, 고객 평점 같은 정보가 들어 있을 수 있죠.\n   - **임베딩**이라는 것을 통해 컴퓨터가 이 줄들을 이해할 수 있게 만들어 줘요. 임베딩은 쉽게 말해, 각 줄을 컴퓨터가 알아들을 수 있는 숫자들의 묶음으로 바꾸는 거예요.\n\n2. **표에서 임베딩 만들기**:\n   - 표의 각 줄을 하나의 긴 문장으로 바꿔서, 컴퓨터가 이해할 수 있는 방법으로 변환해요(임베딩).\n   - `create_table_embeddings`라는 함수가 이 일을 해줘요.\n\n3. **질문에 따라 비슷한 줄 찾기**:\n   - 누군가 질문을 하면, 그 질문도 임베딩으로 바꿔요.\n   - 그런 다음, 그 질문과 가장 비슷한 줄을 표에서 찾아내요.\n   - `find_relevant_rows`라는 함수가 이 일을 해주죠. 예를 들어, "어떤 제품이 가장 높은 고객 평점을 받았나요?"라는 질문을 하면, 평점이 관련된 줄을 찾는 거예요.\n\n4. **질문에 답하기**:\n   - 질문과 관련 있는 줄들을 모아서 보여줘요.\n   - `answer_table_question`라는 함수가 최종적으로 관련 있는 줄들을 찾아줘서, 우리가 답을 알 수 있도록 도와줍니다.\n\n그래서 궁극적으로, 이 코드는 표의 정보를 잘 이해해서, 우리가 하는 질문에 답할 수 있게 도와주는 프로그램이랍니다!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
